In [1]:
import PIL
import numpy as np
import torch
import torchvision
from torchvision.transforms.v2 import Compose, GaussianBlur, RandomEqualize, RandomSolarize, RandomApply
import wandb
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset
from tasks.SRDiffTrainer import SRDiffTrainer
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

#Data
lr_size = 64
hr_size = 256
batch_size = 20
dataset_dir = 'C:\\Users\\adrianperera\\Desktop\\dataset_tfg'

transforms = Compose(
    [RandomApply(transforms= [GaussianBlur(7)], p = 0.5),
    RandomEqualize()]
)

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation = None, aux_sat_prob= 0, sat_dataset_path= "C:\\Users\\adrianperera\\Desktop\\dataset_tfg\\satelite_dataset\\64_256")
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device(0)

In [2]:
model_builder = SRDiffBuilder()
model = model_builder.build_standart()
model.to(device)

GaussianDiffusion(
  (denoise_fn): Unet(
    (cond_proj): ConvTranspose2d(96, 64, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
    (time_pos_emb): SinusoidalPosEmb()
    (mlp): Sequential(
      (0): Linear(in_features=64, out_features=256, bias=True)
      (1): Mish()
      (2): Linear(in_features=256, out_features=64, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0): ResnetBlock(
          (mlp): Sequential(
            (0): Mish()
            (1): Linear(in_features=64, out_features=64, bias=True)
          )
          (block1): Block(
            (block): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
              (2): Mish()
            )
          )
          (block2): Block(
            (block): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
              (2): Mish()
            )
 

In [3]:
torch.cuda.empty_cache()

In [4]:
hyperparams = {
    "lr":0.0002,
    "epochs":20,
    "eta_min":1e-7,
    "decay_steps": 100000,
    "gamma" : 0.5,  
    "model" : "SRDiff",
    "batch_size" : batch_size,
    "ddp": False,
    "grad_acum": 0,
    "use_rrdb":True,
    "fix_rrdb":False,
    "aux_l1_loss":True,
    "aux_perceptual_loss":False,
    "aux_ssim_loss":False
}
hyperparams.update(model_builder.get_hyperparameters())
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams["lr"])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams["decay_steps"], gamma=hyperparams["gamma"])

project_name = "SR model benchmarking"
run_name = "SRDiff standart 20 bach"
wandb.login()
wandb.init(project=project_name, config=hyperparams, name=run_name)
torch.backends.cudnn.benchmark = True

trainer = SRDiffTrainer(metrics_used=("ssim", "psnr"), model_name="SRDiff Standart 20 bach", device=device, 
                        use_rrdb=hyperparams["use_rrdb"], fix_rrdb=hyperparams["fix_rrdb"], aux_ssim_loss=hyperparams["aux_ssim_loss"],
                        aux_l1_loss=hyperparams["aux_l1_loss"], aux_perceptual_loss=hyperparams["aux_perceptual_loss"])
trainer.set_model(model)
trainer.set_optimizer(optimizer)
trainer.set_scheduler(scheduler)
for step in range(hyperparams["epochs"]):
    with torch.no_grad():
        val_loss = trainer.validate(val_dataloader)
    torch.cuda.empty_cache()
    train_loss = trainer.train(train_dataloader)
    torch.cuda.empty_cache()
    if step % 10 == 0:
        trainer.save_model("saved models\\SRDiff")
    torch.cuda.empty_cache()
    wandb.log({"train_loss": train_loss, "validation_loss": val_loss})
    
test_metrics = trainer.test(test_dataloader)
wandb.log(test_metrics)
wandb.finish()

wandb: Currently logged in as: adrianpereramoreno (ladoscuro). Use `wandb login --relogin` to force relogin


100%|██████████| 40/40 [08:28<00:00, 12.72s/batch, psnr=25.6, ssim=0.845]


psnr,▁
ssim,▁
train_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,28.39801
ssim,0.85978
train_loss,0.06948
validation_loss,0.07335
